In [19]:
import pandas as pd
import numpy as np
import requests
import folium
import polyline
import urllib
import matplotlib.pyplot as plt
import seaborn as sns
import time
import math
import json
from branca.element import Figure
from sklearn.cluster import KMeans
from geopy import distance

In [20]:
# retrieve number of vehicles from previous notebook
%store -r available_vehicles 

In [21]:
# import lat lng clustered df

daily_order_df = pd.read_csv('../data/daily_order_df_clustered.csv')
# daily_order_df.drop(labels = 'Unnamed: 0', axis = 1, inplace = True)
df_lat_lng = pd.read_csv('../data/df_lat_lng.csv') # for function route_all_clusters

In [22]:
# create new list to add nested lists for routes

sample_daily_order_list = []
for i in range(available_vehicles):
    sample_daily_order_list.append(daily_order_df[daily_order_df['cluster'] == i]['Name'].tolist())

In [23]:
# saving depot hint

depot_url = 'http://localhost:5000/trip/v1/driving/'
depot_lng = df_lat_lng[df_lat_lng['Name'] == '10 woodlands terrace, singapore']['Lng'].item()
depot_lat = df_lat_lng[df_lat_lng['Name'] == '10 woodlands terrace, singapore']['Lat'].item()
depot_loc = f'{depot_lng},{depot_lat};{depot_lng},{depot_lat}'
r = requests.get(depot_url + depot_loc) 
print(f'Status code: {r.status_code}')
res = r.json()
depot_hint = res['waypoints'][0]['hint']

Status code: 200


In [24]:
# create routing function for sample_daily_order_list

def route_all_clusters(x):
    
    # setting up folium figure
    fig=Figure(width = 900, height = 700)
    m1 = folium.Map(location = ['1.369528', '103.802410'], zoom_start=12)
    folium.Marker(location=['1.4504311', '103.8059588'],icon=folium.Icon(color='black',icon='none'), tooltip = 'Depot').add_to(m1)
    
    # local variables
    cluster_number = 0
    color_num = 0
    color_list = ['lightblue', 'darkblue', 'purple', 'green', 'cadetblue', 'orange', 'lightgreen', 'lightred', 'red', 'darkgreen', 'blue', 'gray', 'beige', 'darkpurple', 'pink', 'lightgray', 'darkred']
        
    # retrieving list from nested list
    for clustered_route in x:
        clustered_route.append('10 woodlands terrace, singapore')

    # creating nested list of coordinates for each delivery location
        locations = []
        for location in clustered_route:
            point = []
            point.append(df_lat_lng['Lng'][df_lat_lng['Name'] == location].item())
            point.append(df_lat_lng['Lat'][df_lat_lng['Name'] == location].item())
            locations.append(point)

    # create string to input all locations for api request
        loc = ''
        for i in range(len(locations)):
            if i != (len(locations) - 1):
                loc += f'{locations[i][0]},{locations[i][1]};'
            else:
                loc += f'{locations[i][0]},{locations[i][1]}'

        url = "http://localhost:5000/trip/v1/driving/"
    
    # api request
        r = requests.get(url + loc) 
        if r.status_code!= 200:
            return {}
        res = r.json()
        res_file = res

    # drawing markers for each delivery location
        for i in range(len(res['waypoints'])):

            if res['waypoints'][i]['hint'] != depot_hint:
                lat = res['waypoints'][i]['location'][1]
                lng = res['waypoints'][i]['location'][0]
                folium.Marker(location=[lat, lng],icon=folium.Icon(color=color_list[color_num],icon='none'), tooltip = clustered_route[i]).add_to(m1)        

    # add to folium map
        folium.PolyLine(
            polyline.decode(res['trips'][0]['geometry']),
            weight=8,
            color=color_list[color_num],
            opacity=1
        ).add_to(m1)
        
    # all durations in mins, unless specified
        duration = round((res['trips'][0]['duration']/60),0)
        distance = round((res['trips'][0]['distance'] / 1000),2)
        duration_distance = round(((res['trips'][0]['distance']/40)*0.06),0)

    # printing of additional details
        print('Vehicle ', cluster_number)
        print('Traveling Duration: ', int(duration/60), 'hrs', int(duration%60), 'mins')
        print('Distance: ',distance, 'km')
        print('Number of Locations: ', len(clustered_route))
        print(color_list[color_num])
        print('duration distance: ', str(int(duration_distance/60)) + 'hrs ' + str(int(duration_distance%60)) + 'mins')
        duration_distance = duration_distance + (len(clustered_route)*10)
        print('duration distance + buffer: ', str(int(duration_distance/60)) + 'hrs ' + str(int(duration_distance%60)) + 'mins')
        print('---\n')
        
        cluster_number += 1
        color_num += 1
        
    # saving folium file to html for use in streamlit
        m1.save(outfile= "../routes/overall_route.html")

    return(m1)

In [25]:
route_all_clusters(sample_daily_order_list)

Vehicle  0
Traveling Duration:  1 hrs 45 mins
Distance:  62.69 km
Number of Locations:  18
lightblue
duration distance:  1hrs 34mins
duration distance + buffer:  4hrs 34mins
---

Vehicle  1
Traveling Duration:  2 hrs 14 mins
Distance:  99.93 km
Number of Locations:  20
darkblue
duration distance:  2hrs 30mins
duration distance + buffer:  5hrs 50mins
---

Vehicle  2
Traveling Duration:  1 hrs 41 mins
Distance:  66.95 km
Number of Locations:  17
purple
duration distance:  1hrs 40mins
duration distance + buffer:  4hrs 30mins
---

Vehicle  3
Traveling Duration:  2 hrs 18 mins
Distance:  83.07 km
Number of Locations:  30
green
duration distance:  2hrs 5mins
duration distance + buffer:  7hrs 5mins
---



In [26]:
def vehicle_cluster(x):
    
    # local variables
    color_num = x
    color_list = ['lightblue', 'darkblue', 'purple', 'green', 'cadetblue', 'orange', 'lightgreen', 'lightred', 'red', 'darkgreen', 'blue', 'gray', 'beige', 'darkpurple', 'pink', 'lightgray', 'darkred']
    
    # adding depot address if not in list
    z = sample_daily_order_list[x]
    if '10 woodlands terrace, singapore' not in z:
        z.append('10 woodlands terrace, singapore')
    
    # creating list of lat and lng for api
    locations = []
    for item in z:
        point = []
        point.append(df_lat_lng['Lng'][df_lat_lng['Name'] == item].item())
        point.append(df_lat_lng['Lat'][df_lat_lng['Name'] == item].item())
        locations.append(point)

    # creating string for api
    loc = ''
    for i in range(len(locations)):
        if i != (len(locations) - 1):
            loc += f'{locations[i][0]},{locations[i][1]};'
        else:
            loc += f'{locations[i][0]},{locations[i][1]}'

    url = "http://localhost:5000/trip/v1/driving/"

    # api request
    r = requests.get(url + loc) 
    if r.status_code!= 200:
        return {}
    res = r.json()

    # setting up folium figure
    fig=Figure(width = 900, height = 700)
    m2 = folium.Map(location = ['1.4504311', '103.8059588'], zoom_start=12)
    folium.Marker(location=['1.4504311', '103.8059588'],icon=folium.Icon(color='black',icon='none'), tooltip = 'Depot').add_to(m2)

    # adding markers to folium figure
    for i in range(len(res['waypoints'])):
        color_list = ['lightblue', 'darkblue', 'purple', 'green', 'cadetblue', 
                                   'white', 'orange', 'lightgreen', 'lightred', 'red', 'darkgreen', 
                                   'blue', 'gray', 'beige', 'darkpurple', 'pink', 'lightgray', 'darkred']

        if res['waypoints'][i]['hint'] != depot_hint:
            lng = res['waypoints'][i]['location'][1]
            lat = res['waypoints'][i]['location'][0]
            folium.Marker(location=[lng, lat],icon=folium.Icon(color=color_list[color_num],icon='none'), tooltip = z[i]).add_to(m2)


    folium.PolyLine(
        polyline.decode(res['trips'][0]['geometry']),
        weight=8,
        color=color_list[color_num],
        opacity=1
    ).add_to(m2)

    
    # all durations in mins, unless specified
    duration = round((res['trips'][0]['duration']/60),0)
    distance = round((res['trips'][0]['distance'] / 1000),2)
    duration_distance = round(((res['trips'][0]['distance']/40)*0.06),0)

    # printing of additional details
    print('Vehicle ', x)
    print('Traveling Duration: ', int(duration/60), 'hrs', int(duration%60), 'mins')
    print('Distance: ',distance, 'km')
    print('Number of Locations: ', len(sample_daily_order_list[x]))
    print(color_list[color_num])
    print('duration distance: ', str(int(duration_distance/60)) + 'hrs ' + str(int(duration_distance%60)) + 'mins')
    duration_distance = duration_distance + (len(sample_daily_order_list[x])*10)
    print('duration distance + buffer: ', str(int(duration_distance/60)) + 'hrs ' + str(int(duration_distance%60)) + 'mins')
    print('---\n')
    
    # saving folium figure as html file for use with streamlit
    m2.save(outfile= "../routes/overall_route.html")
    
    vehicle_cluster_df = pd.DataFrame()
    


    return m2

In [27]:
vehicle_cluster(0)

Vehicle  0
Traveling Duration:  1 hrs 45 mins
Distance:  62.69 km
Number of Locations:  18
lightblue
duration distance:  1hrs 34mins
duration distance + buffer:  4hrs 34mins
---

